# Web-Scraping. Семинар

Итак, у нас есть код, который собирает ссылки на страницы всех преподавателей, которые ведут у второго курса:

In [52]:
import requests
from bs4 import BeautifulSoup

link = 'https://www.hse.ru/ba/we/tutors' 
page = requests.get(link) 

soup = BeautifulSoup(page.text) 

links = set() 

for person in soup.find_all('div', {'class':"b-greetings__item b-greetings__tutor"}):
    course_info = person.find('div', {'class':"b-greetings__descr small"})
    if '2-й курс' in course_info.get_text():
        try:
            person_info = person.find('div', {'class':"b-greetings__person_data small"})
            links.add('https://www.hse.ru' + person_info.find('a').get('href'))
        except:
            print('Не удалось получить ссылку')
    
print(len(links))

Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
Не удалось получить ссылку
199


Проверим, что у нас нет ссылок, которые ведут не на личную страничку.

In [53]:
to_remove = set()

for link in links:
    if 'persons' not in link: # если в ссылке нет подстроки `persons`, то добавь ее в множество кандидатов на удаление
        to_remove.add(link)

print(to_remove)

{'https://www.hse.ru/best'}


Удалим странички через разность множеств.

In [54]:
links = links - to_remove

Чтобы ускорить процесс, будем скачиывать файлы и считать статистику для первых 10 преподавателей, а потом можем запустить код для всех.

In [56]:
first_ten = sorted(links)[:10] # отсортируем, чтобы работать с одинаковыми страничками
print(first_ten)

['https://www.hse.ru/mirror/org/persons/redir/101530936?pl=', 'https://www.hse.ru/mirror/org/persons/redir/101534274?pl=', 'https://www.hse.ru/mirror/org/persons/redir/101539618?pl=', 'https://www.hse.ru/mirror/org/persons/redir/101555405?pl=', 'https://www.hse.ru/mirror/org/persons/redir/102646386?pl=', 'https://www.hse.ru/mirror/org/persons/redir/10443162?pl=', 'https://www.hse.ru/mirror/org/persons/redir/10444528?pl=', 'https://www.hse.ru/mirror/org/persons/redir/10445847?pl=', 'https://www.hse.ru/mirror/org/persons/redir/105131657?pl=', 'https://www.hse.ru/mirror/org/persons/redir/105604470?pl=']


Попробуем скачать резюме для какого-нибудь преподавателя из нашего списка.

In [57]:
page = requests.get(first_ten[1]).text
print('Резюме' in page) # проверяем, есть ли на странице ссылка на резюме

True


In [58]:
# ссылка на резюме есть, поэтому продолжим обработку страницы
soup_link = BeautifulSoup(page)

# находим нужную ссылку по тексту
print(soup_link.find_all('a', text='Резюме')) 

# достаем сам адрес ссылки на резюме
print(soup_link.find_all('a', text='Резюме')[0].get('href')) 

# восстанавливаем полный адрес ссылки на резюме
cv_link = 'https://www.hse.ru' + soup_link.find_all('a', text='Резюме')[0].get('href')

# достаем имя сотрудника для того, чтобы назвать файл
name = soup_link.find('h1').get_text()
print(name)

# обращаемся по ссылке к файлу
doc = requests.get(cv_link)

# проверяем расширение файла, который хранится по ссылке
print(doc.headers['Content-Type'].split('/')[1] == 'pdf')


# создаем файл в режиме записи бинарной информации
with open(f'{name}.pdf', 'wb') as fh:
    # записываем в файл содержание файла по ссылке в байтах
    fh.write(doc.content)

[<a class="link" href="/data/2019/10/30/391720357032193/CV.pdf" target="_blank">Резюме</a>]
/data/2019/10/30/391720357032193/CV.pdf
Хажгериева Анастасия Игоревна
True


Теперь давайте напишем функцию и применим ее ко всем ссылкам в нашем списке.

In [59]:
def get_CV(link):
    page = requests.get(link).text
    if 'Резюме' in page:
        soup_link = BeautifulSoup(page, 'lxml')
        cv_link = 'https://www.hse.ru' + soup_link.find_all('a', text='Резюме')[0].get('href')
        doc = requests.get(cv_link)
        if doc.headers['Content-Type'].split('/')[1] == 'pdf':
            name = soup_link.find('h1').get_text()
            with open(f'{name}.pdf', 'wb') as fh:
                fh.write(doc.content)

In [60]:
for link in first_ten:
    get_CV(link)

## Самостоятельное задание: 
На странице сотрудников есть информация о владении иностранными языками. Для сотрудников факультета создайте словарь, где ключом будет язык, а значением - количество сотрудников, которые указали, что владеют им. Выведите ключи и значения словаря, отсортированные по значениям от большего к меньшему.

In [61]:
import time
import random 

lang_dict = {}
cnt = 0

for link in first_ten:
    try:
        time.sleep(random.randint(1,5))
        page = requests.get(link).text
        soup_link = BeautifulSoup(page, 'lxml')
        lang = soup_link.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0]
        for l in lang.find_all('dd'):
            l_text = l.get_text()
            lang_dict[l_text] = lang_dict.get(l_text, 0) + 1
    except:
        pass
        
print(sorted(lang_dict.items(), key = lambda x: x[0]), end='\n')

[('английский', 8), ('испанский', 2), ('итальянский', 1), ('немецкий', 2), ('русский', 1), ('украинский', 1), ('французский', 2), ('шведский', 1)]


А теперь запустим для всех. Внимание, этот код может легко выполняться 5-15 минут.

In [62]:
import time
import random 

lang_dict = {}
cnt = 0

for link in links: # проходим по множеству всех наших преподавателей
    cnt += 1
    if cnt % 10 == 0:
        print(f'Обрабатываю ссылку №{cnt} из {len(links)}') # добавим счетчик, чтобы у нас печатался своеобразный progress bar
    try:
        time.sleep(random.randint(1,5))
        page = requests.get(link).text
        soup_link = BeautifulSoup(page, 'lxml')
        lang = soup_link.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0]
        for l in lang.find_all('dd'):
            l_text = l.get_text()
            lang_dict[l_text] = lang_dict.get(l_text, 0) + 1
    except:
        pass
        
print(sorted(lang_dict.items(), key = lambda x: x[0]), end='\n')

Обрабатываю ссылку №50 из 198
Обрабатываю ссылку №100 из 198
Обрабатываю ссылку №150 из 198
[('английский', 146), ('английский (Преподаю английский)', 1), ('английский (продвинутый уровень)', 1), ('английский (профессиональное владение)', 1), ('английский (свободное владение/C2)', 1), ('арабский', 2), ('древнеанглийский', 1), ('древнегреческий', 1), ('древнеегипетский', 1), ('иврит', 2), ('испанский', 21), ('итальянский', 10), ('итальянский (CILS C1)', 1), ('китайский', 5), ('латинский', 3), ('литовский', 1), ('немецкий', 59), ('немецкий (Изучала немецкий в университете)', 1), ('немецкий (средний/A2.2)', 1), ('польский', 4), ('португальский', 4), ('румынский', 1), ('румынский (молдавский)', 1), ('русский', 6), ('русский (профессиональный преподаватель РКИ)', 1), ('русский (родной язык)', 1), ('сербский', 1), ('узбекский', 1), ('украинский', 2), ('финский', 1), ('французский', 41), ('французский (начальный/A2.1)', 1), ('французский (средний уровень)', 1), ('хорватский', 1), ('шведский',

Следующим пунктом будет решить проблему, как избавиться от уточнений вида `(продвинутый уровень)` и так далее.